# Import Neccesary packages

In [1]:
#All neccesary classes for project

#general
import seaborn as sns
import scipy.stats as ss
import pandas as pd
import numpy as np
import os
import pickle
import time
import tensorflow as tf
import shutil
from datetime import datetime
import os
import random
import sys
import threading


#for utility packages
from Utilities.utilities import import_data
from Utilities.utilities import DataFrameSelector
from Utilities.utilities import CategoricalEncoder

from functools import partial

#image manipulation
from PIL import Image as PI
from PIL import ImageFilter
from resizeimage import resizeimage
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#Import Custom Functions
from Utilities.model_builder import get_image
from Utilities.model_builder import get_file_lists
from Utilities.model_builder import parse_record
from Utilities.model_builder import get_batch
from Utilities.model_builder import build_iterator
from Utilities.models import log_dir_build
from Utilities.utilities import generate_image
from Utilities.utilities import generate_image_array
from Utilities.blur_tool import blur_images_in_directory
from Utilities.resize_tool import resize_images_in_directory
from Utilities.blur_tool import blur_and_print_image
from Utilities.utilities import import_data
from Utilities.utilities import DataFrameSelector
from Utilities.utilities import CategoricalEncoder
from Utilities.utilities import display_scores
from Utilities.bounded_file_label_extractor import get_files_and_labels
from Utilities.bounded_box_record_maker import process_bounded_image_files

#for preprocessing
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.utils import shuffle

# Import the data and split into training and validation data

Here we are going to import the raw data and create a training and validation set

In [2]:
raw_data_csv = "image_table_data.csv"
data_folder = "D:/Machine_Learning/Datasets/corpus_learning"
image_folder = "D:/Machine_Learning/Datasets/corpus_learning/raw_data/"
tf_record_folder = "D:/Machine_Learning/Datasets/corpus_learning/tf_records_2/"
 
train_folder = "D:/Machine_Learning/Datasets/corpus_learning/training_data/"
train_csv = "D:/Machine_Learning/Datasets/corpus_learning/" + "train_data.csv"
val_folder = "D:/Machine_Learning/Datasets/corpus_learning/validation_data/"
val_csv = "D:/Machine_Learning/Datasets/corpus_learning/" + "val_data.csv"

In [ ]:
raw_data = import_data(raw_data_csv, data_folder)
raw_data

In [ ]:
#Split data by type
train = shuffle(raw_data.iloc[0:1728])
test = shuffle(raw_data.iloc[1729::])
#print the CSV into desired folders
train.to_csv(train_csv)
test.to_csv(val_csv)

In [ ]:
#Split data into train and test sets
train, test = train_test_split(raw_data, test_size=0.25, random_state=42)
train = train.reset_index().drop(['index'], axis=1)
test = test.reset_index().drop(['index'], axis=1)

#print the CSV into desired folders
train.to_csv(train_csv)
test.to_csv(val_csv)

In [ ]:
#move all corresponding images to correct test and validation folder

#move training data
for x in train['Filename']:
    shutil.move(image_folder + x, train_folder + x)
    
#move validation data
for x in test['Filename']:
    shutil.move(image_folder + x, val_folder + x)


# Make Tensorflow Records

Here we are going to make tfrecords using the data we just divided up

In [3]:
train = import_data(train_csv,'').drop(['Unnamed: 0'], axis=1)
val = import_data(val_csv, '').drop(['Unnamed: 0'], axis=1)
train

,Filename,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,corpus_image_original_17_angle_35.jpg,142.993710,144.597049,158.934391,117.208556,142.692736,152.607486,106.457284,156.721575
1,corpus_image_original_3_angle_10.jpg,123.998524,138.526647,152.884963,131.643411,118.991360,144.721916,90.685235,128.586707
2,corpus_image_original_24_angle_190.jpg,118.243875,114.239203,86.902136,129.500445,123.831078,110.237817,159.495678,120.476751
3,corpus_image_original_2_angle_230.jpg,101.491180,103.129329,95.701644,131.592177,99.418341,96.263624,126.554203,80.012108
4,corpus_image_original_4_angle_355.jpg,123.324984,150.708503,161.799615,148.822646,115.542128,153.786036,88.809240,119.450943
5,corpus_image_original_1_angle_130.jpg,148.953492,113.791082,121.733644,95.493240,155.853843,116.559261,152.666862,156.019370
6,corpus_image_original_19_angle_35.jpg,138.442519,123.488662,156.810457,93.134998,136.957475,131.609942,95.840477,134.491928
7,corpus_image_original_24_angle_10.jpg,137.756125,141.760797,169.097864,126.499555,132.168923,145.762183,96.504322,135.523249
8,corpus_image_original_22_angle_340.jpg,109.119896,145.711057,143.404341,154.021612,100.628248,146.871059,89.463913,106.720981
9,corpus_image_original_21_angle_200.jpg,120.852216,119.137526,92.728688,145.197079,124.806689,112.198496,167.656303,118.302793


In [4]:
train_files, train_labels = get_files_and_labels(train, train_folder,'Filename')
val_files, val_labels = get_files_and_labels(val, val_folder,'Filename')

In [5]:
#Make train_records
process_bounded_image_files("train", train_files, train_labels, 2, 2, tf_record_folder)

Launching 2 threads for spacings: [[0, 864], [864, 1728]]
2018-11-26 17:08:18.326812 [thread 0]: Wrote 864 images to D:/Machine_Learning/Datasets/corpus_learning/tf_records_2/train-00000-of-00002
2018-11-26 17:08:18.345812 [thread 0]: Wrote 864 images to 864 shards.
2018-11-26 17:08:18.345812 [thread 1]: Wrote 864 images to D:/Machine_Learning/Datasets/corpus_learning/tf_records_2/train-00001-of-00002
2018-11-26 17:08:18.347812 [thread 1]: Wrote 864 images to 864 shards.
2018-11-26 17:08:18.516811: Finished writing all 1728 images in data set.


In [6]:
#Make test_records
process_bounded_image_files("validation", val_files, val_labels, 2, 2, tf_record_folder)


Launching 2 threads for spacings: [[0, 287], [287, 575]]
2018-11-26 17:08:19.368810 [thread 0]: Wrote 287 images to D:/Machine_Learning/Datasets/corpus_learning/tf_records_2/validation-00000-of-00002
2018-11-26 17:08:19.369814 [thread 0]: Wrote 287 images to 287 shards.
2018-11-26 17:08:19.387811 [thread 1]: Wrote 288 images to D:/Machine_Learning/Datasets/corpus_learning/tf_records_2/validation-00001-of-00002
2018-11-26 17:08:19.389813 [thread 1]: Wrote 288 images to 288 shards.
2018-11-26 17:08:19.558811: Finished writing all 575 images in data set.


# Dev Area

Use this area to run tests on above